In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from numpy.linalg import lstsq
import matplotlib.pyplot as plt


Data Reading Statements

In [3]:
portfolios25 = pd.read_csv('25_Portfolios.CSV', index_col=0)
mom = pd.read_csv('Momentum.csv', index_col=0)
ff3 = pd.read_csv('3_Factor.csv', index_col=0)
ff5 = pd.read_csv('5_Factor.csv', index_col=0)
rev = pd.read_csv('Reversal.csv', index_col=0)

portfolios25.index = pd.to_datetime(portfolios25.index, format='%Y%m')
mom.index = pd.to_datetime(mom.index, format='%Y%m')
ff3.index = pd.to_datetime(ff3.index, format='%Y%m')
ff5.index = pd.to_datetime(ff5.index, format='%Y%m')
rev.index = pd.to_datetime(rev.index, format='%Y%m')

portfolios25 = portfolios25.loc['1963-07-01':'2024-06-01']
mom = mom.loc['1963-07-01':'2024-06-01']
ff3 = ff3.loc['1963-07-01':'2024-06-01']
ff5 = ff5.loc['1963-07-01':'2024-06-01']
rev = rev.loc['1963-07-01':'2024-06-01']

ff3_mom = pd.concat([ff3, mom], axis=1)
ff3_rev = pd.concat([ff3, rev], axis=1)

CONS = pd.Series(np.ones(len(ff3_mom)), index=ff3_mom.index)
CONS.name = "CONS"

ff3_mom = pd.concat([ff3_mom, CONS], axis=1)
ff3_rev = pd.concat([ff3_rev, CONS], axis=1)
ff3 = pd.concat([ff3, CONS], axis=1)
ff5 = pd.concat([ff5, CONS], axis=1)

FileNotFoundError: [Errno 2] No such file or directory: 'Momentum.csv'

Part (a) regressions - Time Series

In [ ]:
def grsstat(T,N,K,model):
    alphas = pd.Series()
    matrix = pd.DataFrame()
    for i in range(0,len(portfolios25.columns)):
        y = portfolios25.iloc[:,i] - model.loc[:,'RF']
        x = model.drop('RF', axis=1)
        mod = sm.OLS(y, x)
        res = mod.fit()
        alphas[i] = res.params.loc["CONS"]
        matrix[portfolios25.columns[i]] = res.resid

    alphas = alphas.to_frame()
    for j in range(0,len(matrix)):
        resid1 = matrix.iloc[j].to_frame() @ matrix.iloc[j].to_frame().transpose()
        if j == 0:
            sigmasum = resid1
        else:
            sigmasum = sigmasum + resid1
    sigma = sigmasum/732
    sigmainverse = np.linalg.inv(sigma)
    factor = model.drop(['RF','CONS'], axis=1)
    for g in range(0,len(matrix)):
        diff = factor.iloc[g] - factor.mean()
        covar = diff.to_frame() @ diff.to_frame().transpose()
        if g == 0:
            omegasum = covar
        else:
            omegasum = omegasum + covar

    omega = omegasum/732
    omegainverse = np.linalg.inv(omega)

    factor_mean = factor.mean().to_frame()
    temp = factor_mean.T @ omegainverse
    temp.columns = factor_mean.index
    lhs = (temp @ factor_mean + 1) ** -1
    rhs = alphas.T @ sigmainverse @ alphas
    grs = (((T-N-K)/N) * lhs * rhs).iloc[0,0]
    print(f"The GRS stat for this model is {grs}")

grsstat(732,25,5,ff5)
grsstat(732,25,4,ff3_mom)
grsstat(732,25,4,ff3_rev)


The GRS stat for this model is 3.1765811560751165
The GRS stat for this model is 3.3432127684005932
The GRS stat for this model is 4.047110643600798


In [ ]:
def grsstatalt(T,N,K,model):
    alphas = pd.Series()
    matrix = pd.DataFrame()
    for i in range(0,len(portfolios25.columns)):
        y = portfolios25.iloc[:,i] - model.loc[:,'RF']
        x = model.drop('RF', axis=1)
        mod = sm.OLS(y, x)
        res = mod.fit()
        alphas[i] = res.params.loc["CONS"]
        matrix[portfolios25.columns[i]] = res.resid

    alphas = alphas.to_frame()
    sigma = np.cov(matrix, rowvar=False, bias=True)
    sigmainverse = np.linalg.inv(sigma)
    factor = model.drop(['RF','CONS'], axis=1)
    omega = np.cov(factor, rowvar=False, bias=True)
    omegainverse = np.linalg.inv(omega)

    factor_mean = factor.mean().to_frame()
    temp = factor_mean.T @ omegainverse
    temp.columns = factor_mean.index
    lhs = (temp @ factor_mean + 1) ** -1
    rhs = alphas.T @ sigmainverse @ alphas
    grs = (((T-N-K)/N) * lhs * rhs).iloc[0,0]
    print(f"The GRS stat for this model is {grs}")

grsstatalt(732,25,5,ff5)
grsstatalt(732,25,4,ff3_mom)
grsstatalt(732,25,4,ff3_rev)


The GRS stat for this model is 3.176581156075117
The GRS stat for this model is 3.343212768400592
The GRS stat for this model is 4.047110643600795


In [ ]:
tony = pd.read_csv("merge_d.csv")

NameError: name 'pd' is not defined

Part (c) regressions - Cross Sectional

In [ ]:
#Defining Cross Sectional Function
def fama1973test(T, model):
    #First Pass
    first = sm.OLS(portfolios25.sub(model['RF'],axis = 0),model.drop(['RF'], axis=1))
    firstres = first.fit()
    betas = firstres.params
    betas.columns = portfolios25.columns
    betas = betas.T
    betas["CONS"] = np.ones(len(betas))

    #Second Pass
    second = sm.OLS(portfolios25.sub(model['RF'],axis = 0).T, betas)
    secondres = second.fit()
    means = secondres.params.mean(axis=1)
    se = secondres.params.sub(means,axis = 0) **2
    sse = se.sum(axis=1)
    var = sse/(T**2)
    t_stat = means/(var**(1/2))
    factors = model.drop(['RF','CONS'], axis=1)
    factors_mean = factors.mean()
    factors_var = factors.var()
    correction = (factors_mean - means["CONS"])**2/factors_var +1
    correction["CONS"] = 1
    t_stat_corrected = means/((var*correction)**(1/2))
    print(f"The t-statistic for this model :"
          f" {t_stat}")
    print(f"The corrected t-statistic for this model is "
          f"{t_stat_corrected}")

fama1973test(732,ff5)
fama1973test(732,ff3_mom)
fama1973test(732,ff3_rev)



The t-statistic for this model : Mkt-RF   -1.095852
SMB       2.245241
HML       2.376790
RMW       2.847728
CMA       0.303824
CONS      3.293718
dtype: float64
The corrected t-statistic for this model is Mkt-RF   -1.093230
SMB       2.188454
HML       2.329012
RMW       2.748875
CMA       0.290773
CONS      3.293718
dtype: float64
The t-statistic for this model : Mkt-RF   -0.044209
SMB       0.916669
HML       3.130605
Mom       3.405397
CONS      2.270480
dtype: float64
The corrected t-statistic for this model is Mkt-RF   -0.044204
SMB       0.905158
HML       3.107085
Mom       3.405232
CONS      2.270480
dtype: float64
The t-statistic for this model : Mkt-RF   -1.735135
SMB       1.030179
HML       3.032371
ST_Rev   -2.867725
CONS      4.318588
dtype: float64
The corrected t-statistic for this model is Mkt-RF   -1.721281
SMB       0.979988
HML       2.912165
ST_Rev   -2.798297
CONS      4.318588
dtype: float64
